In [73]:
import pandas as pd

df = pd.read_csv('tools-for-thought.csv')
sample = df.sample(100)


In [96]:
entities = []
for k, v in sample['entities.annotations'].items():
    tags = 
    entities.append(tags)


ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [58]:
# mark standalone, reply, rt
sample['rt'] = sample["referenced_tweets.retweeted.id"].isna().apply(lambda x: not x)
sample['reply'] = sample["referenced_tweets.replied_to.id"].isna().apply(lambda x: not x)
sample['quote'] = sample["referenced_tweets.quoted.id"].isna().apply(lambda x: not x)
sample['standalone'] = df[["referenced_tweets.replied_to.id",	"referenced_tweets.retweeted.id", "referenced_tweets.quoted.id"]].isna().sum(axis=1).apply(lambda x: True if x == 3 else False)

# rename author.name to name
sample = sample.rename(columns={'author.name': 'name', 'author.username': 'username'})

# find all columns starting with 'public metrics'
names = [col for col in df.columns if col.startswith('public_metrics')]
# remove 
new_names = [col.split('.')[1] for col in names]
sample = sample.rename(columns=dict(zip(names, new_names)))

In [59]:
# Convert to desired schema

sample['id'] = sample['id'].astype(str)
sample['author'] = sample[['name', 'username']].to_dict(orient='records')
sample['html'] = sample['text'].astype(str)
sample['public_metrics'] = sample[new_names].to_dict(orient='records')

sample = sample[['id', 'author', 'html', 'created_at', 'public_metrics', 'standalone', 'rt', 'reply', 'quote'] + ['geo.country', 'entities.annotations', 'context_annotations']]

sample.to_json('sample.json', orient='records')

In [60]:
sample

,id,author,html,created_at,public_metrics,standalone,rt,reply,quote,geo.country,entities.annotations,context_annotations
656,1594360915411243008,"{'name': 'ArchaeoHistories', 'username': 'histories_arch'}","This is one of the oldest buildings in Hattfjelldal municipality in Norland, Norway. Built in the 18th Century CE.\n\n#archaeohistories https://t.co/ojx7jMGzS8",2022-11-20T16:03:58.000Z,"{'like_count': 10548, 'quote_count': 317, 'reply_count': 207, 'retweet_count': 1600}",True,False,False,False,NaN,"[{""start"": 39, ""end"": 63, ""probability"": 0.7593, ""type"": ""Place"", ""normalized_text"": ""Hattfjelldal municipality""}, {""start"": 68, ""end"": 74, ""probability"": 0.7252, ""type"": ""Place"", ""normalized_text"": ""Norland""}, {""start"": 77, ""end"": 82, ""probability"": 0.894, ""type"": ""Place"", ""normalized_text"": ""Norway""}]",NaN
873,1593722841626824707,"{'name': 'paradox 🍁', 'username': 'zeno_dox'}",@codexeditor You can hear the Liszt in Nyiregyhazi’s performance.,2022-11-18T21:48:30.000Z,"{'like_count': 1, 'quote_count': 0, 'reply_count': 0, 'retweet_count': 1}",False,False,True,False,NaN,"[{""start"": 30, ""end"": 34, ""probability"": 0.8979, ""type"": ""Person"", ""normalized_text"": ""Liszt""}, {""start"": 39, ""end"": 49, ""probability"": 0.4281, ""type"": ""Person"", ""normalized_text"": ""Nyiregyhazi""}]",NaN
1116,1593310583914471429,"{'name': 'The Syllabus', 'username': 'syllabus_tweets'}",El Rencor de la Clase Media y el Fin de una Era\n\nBy @HdezEsteban and @JDSato in @AkalEditor\n\nhttps://t.co/BnEouqMGIs,2022-11-17T18:30:20.000Z,"{'like_count': 5, 'quote_count': 0, 'reply_count': 0, 'retweet_count': 3}",True,False,False,False,NaN,"[{""start"": 3, ""end"": 26, ""probability"": 0.5272, ""type"": ""Other"", ""normalized_text"": ""Rencor de la Clase Media""}, {""start"": 33, ""end"": 46, ""probability"": 0.6348, ""type"": ""Other"", ""normalized_text"": ""Fin de una Era""}]",NaN
488,1594821810859778051,"{'name': 'Napkin', 'username': 'napkin_ideas'}","@dltnio With tools like @napkin_ideas, we’re already seeing AI be used to make 2 go faster, and it’s hard not to see this exist as a feature in every note taking tool (e.g. Notion AI or Mem AI). \n\nThat will take this from a “habit only crazy people without kids” can do to “everyone can”",2022-11-21T22:35:24.000Z,"{'like_count': 3, 'quote_count': 0, 'reply_count': 0, 'retweet_count': 1}",False,True,False,False,NaN,"[{""start"": 60, ""end"": 61, ""probability"": 0.6824, ""type"": ""Organization"", ""normalized_text"": ""AI""}, {""start"": 180, ""end"": 181, ""probability"": 0.5135, ""type"": ""Organization"", ""normalized_text"": ""AI""}, {""start"": 186, ""end"": 191, ""probability"": 0.4601, ""type"": ""Other"", ""normalized_text"": ""Mem AI""}]",NaN
1057,1593270020158541824,"{'name': 'Hourly Seventh Doctor Era', 'username': 'HourlySeventh'}",https://t.co/grsBBkfvl1,2022-11-17T15:49:08.000Z,"{'like_count': 24, 'quote_count': 2, 'reply_count': 0, 'retweet_count': 1}",True,False,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1045,1593330146651430912,"{'name': 'Remedios Varo', 'username': 'artistvaro'}",The task #varo #surrealism https://t.co/FGfUCHYAny https://t.co/mOSYuCsBAs,2022-11-17T19:48:04.000Z,"{'like_count': 309, 'quote_count': 3, 'reply_count': 1, 'retweet_count': 55}",True,False,False,False,NaN,"[{""start"": 10, ""end"": 13, ""probability"": 0.3724, ""type"": ""Other"", ""normalized_text"": ""varo""}, {""start"": 16, ""end"": 25, ""probability"": 0.5819, ""type"": ""Other"", ""normalized_text"": ""surrealism""}]",NaN
586,1594570256835805184,"{'name': 'Logseq 🪵', 'username': 'logseq'}",🥳 https://t.co/TLYCRXV3kb,2022-11-21T05:55:49.000Z,"{'like_count': 27, 'quote_count': 0, 'reply_count': 1, 'retweet_count': 0}",False,False,False,True,NaN,NaN,NaN
680,1594177787086409728,"{'name': 'Xrine 蝕鈴', 'username': 'xrinexrine'}",沈迷Heptabase⋯⋯\n你喜歡寫設定嗎？需要精神時光屋嗎？\n假日醒來直接開，一天就會消失不見ㄛ！\n欸不是我的初衷明明就是拿來整理思路想策略，結果失控一直爆寫文章草稿（我以為我已經沒有力量寫部